In [1]:
import numpy as np
import cv2
import sys
import os
import glob
import xml.etree.ElementTree as ET
import scipy
import scipy.misc
from lxml import etree

# for calling the haarcascade xml file
cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)


In [2]:
# enter the location for creating a new database
new_database = r'D:\face detection and recognition\alok\FaceVOC' 
if not os.path.exists(new_database):
    os.makedirs(new_database)
count = 0
ii=0
#enter the number of classes ( Here the number of videos named as videox (where x should be 0,1,2,3...num_class-1)
# should be equal to number of classes,where each video bears each class)
num_class=4
#enter the name of the classes
name_class=["name1","name2","name3","name4"]
for ii in range(0,num_class):
    
    #Enter the address of the video location or enter 0 for using webcam usage,
    # the program can be modified accordingly by closing the present webcam window and opening a new one at the end of each loop.
    vidcap = cv2.VideoCapture('D:\\face detection and recognition\\alok\\videos\\video%d.mp4'%ii)
    success,image = vidcap.read()
    
    success = True
    #count+=count
    while success:
        #enter the factor by which the width and height have to be divided. For keeping original image, keep factor as 1
        factor=2.56
        a=int(image.shape[1]/2.56)  
        b=int(image.shape[0]/2.56)
    
        frame=cv2.resize(image, (a,b))
        
        #create the folder where the image files has to be saved
        destinationfolder = os.path.join(new_database,"JPEGImages")
        if not os.path.exists(destinationfolder):
            os.makedirs(destinationfolder)
        cv2.imwrite(os.path.join(destinationfolder,"Image%d.jpg" % count), frame)     # save frame as JPEG file
  
        # divide the images for training, test and validation
        destinationdivide = os.path.join(new_database,"Main")
        if not os.path.exists(destinationdivide):
            os.makedirs(destinationdivide)
        

        x=None
        y=None
        w=None
        h=None              
        # Obtain coordinates of face

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), 
                                        flags=cv2.CASCADE_SCALE_IMAGE)
        
        
        # Draw a rectangle around the faces
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        if (x!=None) and (y!=None) and (h!=None) and (w!=None):
            f1= ("Image%d\n" %count)
            
            # the number of images you need for training validation etc .. can be varied here by varying the number after % 
            # as per your choice
            
            
            
            if (count % 2) == 0:
                with open(os.path.join(destinationdivide,"trainval.txt"),"a") as ufile1: 
                        ufile1.write(f1)
    
            else:
                with open(os.path.join(destinationdivide,"test.txt"), "a") as ufile2: 
                        ufile2.write(f1)
            
            if(count % 3)== 0:
                with open(os.path.join(destinationdivide,"val.txt"), "a") as ufile3: 
                        ufile3.write(f1)     
            else:
                with open(os.path.join(destinationdivide,"train.txt"), "a") as ufile4: 
                        ufile4.write(f1)
            
            
            
            
        # write to xml file
            destination_annote= os.path.join(new_database,"Annotations")
            if not os.path.exists(destination_annote):
                os.makedirs(destination_annote)
            outFile = open(os.path.join(destination_annote,'Image%d.xml' %count), 'wb')
    
            page = etree.Element('Annotations')

        #use the etree.ElementTree() constructor to make a new document tree, using our html element as its root:

            doc = etree.ElementTree(page)

        #  use etree.SubElement() constructor for adding new child elements to our document. 


            folder = etree.SubElement(page, 'folder')
            folder.text='JPEGImages'

            filename = etree.SubElement(page, 'filename')
            filename.text='Image%d.jpg'%count

            path = etree.SubElement(page, 'path')
            path.text=destinationfolder

            source=etree.SubElement(page, 'source')
            database=etree.SubElement(source, 'database')
            database.text='RECognVOC'

            size=etree.SubElement(page, 'size')
            width=etree.SubElement(size, 'width')
            width.text=str(a)
            height=etree.SubElement(size, 'height')
            height.text=str(b)  
            depth=etree.SubElement(size, 'depth')
            depth.text='3'

            segmented=etree.SubElement(page, 'segmented')
            segmented.text='0'

            object=etree.SubElement(page, 'object')
            name=etree.SubElement(object, 'name')   
        
            name.text=name_class[ii]      
                                       
            pose=etree.SubElement(object, 'pose')
            pose.text='unspecified'
            truncated=etree.SubElement(object, 'truncated')
            truncated.text='0'
            difficult=etree.SubElement(object, 'difficult')
            difficult.text='0'
            bndbox=etree.SubElement(object, 'bndbox')
            xmin=etree.SubElement(bndbox, 'xmin')
            xmin.text=str(x)
            ymin=etree.SubElement(bndbox, 'ymin')
            ymin.text=str(y)
            xmax=etree.SubElement(bndbox, 'xmax')
            xmax.text=str(x+w)
            ymax=etree.SubElement(bndbox, 'ymax')
            ymax.text=str(y+h)
            doc.write(outFile)

        success,image = vidcap.read()
        count += 1  
    
     